In [10]:
import pandas as pd

# Load CSV file
file_path = "D://stocks//PITRO.csv"
df = pd.read_csv(file_path)

# Normalize column names (strip spaces, lowercase)
df.columns = df.columns.str.strip().str.lower().str.replace(" ", "_").str.replace("-", "_")

# Print column names to verify
print("Columns in dataset:", df.columns)

# Map correct column names
column_mapping = {
    "piotroski_score": "f_score",
    "altman_z_score": "z_score",
    "debt_to_equity": "debt_equity",
    "return_on_capital_employed": "roce",
    "return_on_equity": "roe",
    "sales_growth_3years": "growth"
}

# Rename columns
df = df.rename(columns=column_mapping)

# Check for missing columns
required_columns = list(column_mapping.values()) + ["name"]  # Include company name
missing_columns = [col for col in required_columns if col not in df.columns]
if missing_columns:
    print(f"Missing columns: {missing_columns}")
    raise KeyError(f"Missing required columns: {missing_columns}")

# Convert numeric columns to float
for col in ["f_score", "z_score", "debt_equity", "roce", "roe", "growth"]:
    df[col] = pd.to_numeric(df[col], errors='coerce')

# Apply filtering criteria
filtered_df = df[
    (df["f_score"] >= 7) & 
    (df["z_score"] > 4) & 
    (df["debt_equity"] < 1)
].copy()

# Compute ranking score
filtered_df["final_score"] = filtered_df["roce"] + filtered_df["roe"] + filtered_df["growth"]

# Rank and get top 50 stocks
ranked_df = filtered_df.sort_values(by="final_score", ascending=False).head(50)

# Display result
print(ranked_df[["name", "f_score", "z_score", "debt_equity", "roce", "roe", "growth", "final_score"]])

# Save to CSV
output_path = "top_50_stocks.csv"
ranked_df.to_csv(output_path, index=False)
print(f"Top 50 stocks saved as '{output_path}'")

Columns in dataset: Index(['name', 'bse_code', 'nse_code', 'industry', 'current_price',
       'price_to_earning', 'market_capitalization',
       'return_on_capital_employed', 'eps_last_year', 'return_on_equity',
       'book_value', 'price_to_book_value', 'graham_number',
       'historical_pe_7years', 'current_ratio', 'peg_ratio', 'debt_to_equity',
       'altman_z_score', 'piotroski_score', 'npm_last_year',
       'sales_growth_3years', 'eps', 'industry_pe', 'quick_ratio',
       'debtor_days', 'number_of_equity_shares', 'intrinsic_value',
       'enterprise_value', 'return_on_invested_capital'],
      dtype='object')
                 name  f_score  z_score  debt_equity   roce    roe  growth  \
23       Danlaw Tech.      9.0     8.56         0.56  40.21  47.46   96.49   
59     Mefcom Capital      8.0    15.83         0.14  41.14  43.29   96.98   
91    Unique Organics      8.0    10.63         0.07  38.07  32.63   43.41   
3         Adani Power      8.0     4.87         0.67  32.2

In [11]:
df.columns

Index(['name', 'bse_code', 'nse_code', 'industry', 'current_price',
       'price_to_earning', 'market_capitalization', 'roce', 'eps_last_year',
       'roe', 'book_value', 'price_to_book_value', 'graham_number',
       'historical_pe_7years', 'current_ratio', 'peg_ratio', 'debt_equity',
       'z_score', 'f_score', 'npm_last_year', 'growth', 'eps', 'industry_pe',
       'quick_ratio', 'debtor_days', 'number_of_equity_shares',
       'intrinsic_value', 'enterprise_value', 'return_on_invested_capital'],
      dtype='object')

In [19]:
import pandas as pd

# Load CSV file
file_path = "D://stocks//PITRO.csv"
df = pd.read_csv(file_path)

# Normalize column names (strip spaces, lowercase)
df.columns = df.columns.str.strip().str.lower().str.replace(" ", "_").str.replace("-", "_")

# Print column names to verify
print("Columns in dataset:", df.columns)

# Map correct column names
column_mapping = {
    "piotroski_score": "f_score",
    "altman_z_score": "z_score",
    "debt_to_equity": "debt_equity",
    "return_on_capital_employed": "roce",
    "return_on_equity": "roe",
    "sales_growth_3years": "growth",
    "current_ratio": "current_ratio"
}

# Rename columns
df = df.rename(columns=column_mapping)

# Check for missing columns
required_columns = list(column_mapping.values()) + ["name"]  # Include company name
missing_columns = [col for col in required_columns if col not in df.columns]
if missing_columns:
    print(f"Missing columns: {missing_columns}")
    raise KeyError(f"Missing required columns: {missing_columns}")

# Convert numeric columns to float
for col in ["f_score", "z_score", "debt_equity", "roce", "roe", "growth", "current_ratio"]:
    df[col] = pd.to_numeric(df[col], errors='coerce')

# Apply filtering criteria
filtered_df = df[
    (df["f_score"] >= 7) & 
    (df["z_score"] > 4) & 
    (df["debt_equity"] < 1)
].copy()

# Normalize the values between 0 and 1 for fair weighting
filtered_df["debt_equity_norm"] = 1 - (filtered_df["debt_equity"] / filtered_df["debt_equity"].max())  # Lower is better
filtered_df["current_ratio_norm"] = filtered_df["current_ratio"] / filtered_df["current_ratio"].max()
filtered_df["growth_norm"] = filtered_df["growth"] / filtered_df["growth"].max()
filtered_df["roce_norm"] = filtered_df["roce"] / filtered_df["roce"].max()
filtered_df["roe_norm"] = filtered_df["roe"] / filtered_df["roe"].max()

# Assign weightages
weights = {
    "debt_equity_norm": 0.15,
    "current_ratio_norm": 0.20,
    "growth_norm": 0.25,
    "roce_norm": 0.20,
    "roe_norm": 0.20
}

# Compute final weighted score
filtered_df["final_score"] = (
    filtered_df["debt_equity_norm"] * weights["debt_equity_norm"] +
    filtered_df["current_ratio_norm"] * weights["current_ratio_norm"] +
    filtered_df["growth_norm"] * weights["growth_norm"] +
    filtered_df["roce_norm"] * weights["roce_norm"] +
    filtered_df["roe_norm"] * weights["roe_norm"]
)

# Rank and get top 50 stocks
ranked_df = filtered_df.sort_values(by="final_score", ascending=False).head(90)

# Display result
print(ranked_df[["name", "f_score", "z_score", "debt_equity", "current_ratio", "roce", "roe", "growth", "final_score"]])

# Save to CSV
output_path = "top_50_stocks.csv"
ranked_df.to_csv(output_path, index=False)
print(f"Top 50 stocks saved as '{output_path}'")

Columns in dataset: Index(['name', 'bse_code', 'nse_code', 'industry', 'current_price',
       'price_to_earning', 'market_capitalization',
       'return_on_capital_employed', 'eps_last_year', 'return_on_equity',
       'book_value', 'price_to_book_value', 'graham_number',
       'historical_pe_7years', 'current_ratio', 'peg_ratio', 'debt_to_equity',
       'altman_z_score', 'piotroski_score', 'npm_last_year',
       'sales_growth_3years', 'eps', 'industry_pe', 'quick_ratio',
       'debtor_days', 'number_of_equity_shares', 'intrinsic_value',
       'enterprise_value', 'return_on_invested_capital'],
      dtype='object')
                name  f_score  z_score  debt_equity  current_ratio   roce  \
59    Mefcom Capital      8.0    15.83         0.14           6.57  41.14   
87    Sumedha Fiscal      8.0     5.53         0.00          19.93  15.56   
23      Danlaw Tech.      9.0     8.56         0.56           2.37  40.21   
10   Arrow Greentech      8.0    11.62         0.01          1

In [22]:
ranked_df[["name","current_price","f_score","z_score","roce","roe","growth", "debt_equity", "current_ratio","final_score"]].head(40)

,name,current_price,f_score,z_score,roce,roe,growth,debt_equity,current_ratio,final_score
59,Mefcom Capital,16.38,8.0,15.83,41.14,43.29,96.98,0.14,6.57,0.728401
87,Sumedha Fiscal,71.55,8.0,5.53,15.56,12.96,68.54,0.00,19.93,0.625675
23,Danlaw Tech.,974.75,9.0,8.56,40.21,47.46,96.49,0.56,2.37,0.603758
10,Arrow Greentech,522.05,8.0,11.62,35.32,26.07,41.52,0.01,10.71,0.575264
75,R S Software (I),63.97,8.0,7.02,44.35,45.68,18.86,0.01,4.87,0.558063
91,Unique Organics,138.00,8.0,10.63,38.07,32.63,43.41,0.07,4.01,0.532125
47,Kothari Petroche,141.55,8.0,8.25,41.32,28.79,38.54,0.01,3.94,0.529832
26,Dhanalaxmi Roto,210.00,8.0,6.29,38.40,28.43,39.23,0.11,2.69,0.485694
2,Action Const.Eq.,1151.25,8.0,12.67,42.30,30.58,33.41,0.08,1.33,0.484622
22,Chandni Machines,46.40,8.0,10.41,28.91,21.80,56.67,0.01,1.35,0.483357


In [4]:
import requests

# Set your API key
API_KEY = "mmzITKFK4Ac14nBIfjuQLx6lJEn6IC7p"

def get_ticker(company_name):
    """Fetch ticker symbol for a given company name"""
    url = f"https://financialmodelingprep.com/api/v3/search?query={company_name}&limit=1&exchange=NASDAQ&apikey={API_KEY}"
    response = requests.get(url).json()
    if response:
        return response[0]["symbol"]
    return None

def get_company_news(ticker):
    """Fetch latest news for a given ticker"""
    url = f"https://financialmodelingprep.com/api/v3/stock_news?tickers={ticker}&limit=5&apikey={API_KEY}"
    response = requests.get(url).json()
    return response

# Example Usage
company_name = "adani power"
ticker = get_ticker(company_name)

if ticker:
    print(f"Ticker for {company_name}: {ticker}")
    news = get_company_news(ticker)
    
    print("\nLatest News:")
    for article in news:
        print(f"- {article['title']} ({article['publishedDate']})")
        print(f"  {article['url']}\n")
else:
    print(f"No ticker found for {company_name}")

No ticker found for adani power


In [5]:
from yahooquery import search

def get_ticker_yahoo(company_name):
    """Fetch the ticker symbol for a given company name using Yahoo Finance"""
    results = search(company_name)
    if "quotes" in results and results["quotes"]:
        return results["quotes"][0]["symbol"]
    return None

# Example Usage
company_name = "Alkyl Amines"
ticker = get_ticker_yahoo(company_name)

if ticker:
    print(f"Ticker for {company_name}: {ticker}")
else:
    print(f"No ticker found for {company_name}")

Ticker for Alkyl Amines: ALKYLAMINE.NS


In [ ]:
import pandas as pd
from yahooquery import search

# Load CSV file
file_path = "D://stocks//PITRO.csv"
df = pd.read_csv(file_path)

# Normalize column names (strip spaces, lowercase)
df.columns = df.columns.str.strip().str.lower().str.replace(" ", "_").str.replace("-", "_")

# Print column names to verify
print("Columns in dataset:", df.columns)

# Map correct column names
column_mapping = {
    "piotroski_score": "f_score",
    "altman_z_score": "z_score",
    "debt_to_equity": "debt_equity",
    "return_on_capital_employed": "roce",
    "return_on_equity": "roe",
    "sales_growth_3years": "growth",
    "current_ratio": "current_ratio"
}

# Rename columns
df = df.rename(columns=column_mapping)

# Check for missing columns
required_columns = list(column_mapping.values()) + ["name"]  # Include company name
missing_columns = [col for col in required_columns if col not in df.columns]
if missing_columns:
    print(f"Missing columns: {missing_columns}")
    raise KeyError(f"Missing required columns: {missing_columns}")

# Convert numeric columns to float
for col in ["f_score", "z_score", "debt_equity", "roce", "roe", "growth", "current_ratio"]:
    df[col] = pd.to_numeric(df[col], errors='coerce')

# Function to get ticker from company name
def get_ticker_yahoo(company_name):
    results = search(company_name)
    if "quotes" in results and results["quotes"]:
        return results["quotes"][0]["symbol"]
    return None

# Add ticker to dataframe
df["ticker"] = df["name"].apply(get_ticker_yahoo)

# Apply filtering criteria
filtered_df = df[
    (df["f_score"] >= 7) & 
    (df["z_score"] > 4) & 
    (df["debt_equity"] < 1)
].copy()

# Normalize values for scoring
filtered_df["debt_equity_norm"] = 1 - (filtered_df["debt_equity"] / filtered_df["debt_equity"].max())  # Lower is better
filtered_df["current_ratio_norm"] = filtered_df["current_ratio"] / filtered_df["current_ratio"].max()
filtered_df["growth_norm"] = filtered_df["growth"] / filtered_df["growth"].max()
filtered_df["roce_norm"] = filtered_df["roce"] / filtered_df["roce"].max()
filtered_df["roe_norm"] = filtered_df["roe"] / filtered_df["roe"].max()

# Assign weightages
weights = {
    "debt_equity_norm": 0.15,
    "current_ratio_norm": 0.20,
    "growth_norm": 0.25,
    "roce_norm": 0.20,
    "roe_norm": 0.20
}

# Compute final weighted score
filtered_df["final_score"] = (
    filtered_df["debt_equity_norm"] * weights["debt_equity_norm"] +
    filtered_df["current_ratio_norm"] * weights["current_ratio_norm"] +
    filtered_df["growth_norm"] * weights["growth_norm"] +
    filtered_df["roce_norm"] * weights["roce_norm"] +
    filtered_df["roe_norm"] * weights["roe_norm"]
)

# Rank and get top 50 stocks
ranked_df = filtered_df.sort_values(by="final_score", ascending=False).head(50)

# Display result
print(ranked_df[["name", "ticker", "f_score", "z_score", "debt_equity", "current_ratio", "roce", "roe", "growth", "final_score"]])

# Save to CSV
output_path = "top_50_stocks_with_ticker.csv"
ranked_df.to_csv(output_path, index=False)
print(f"Top 50 stocks saved as '{output_path}'")

In [7]:
import pandas as pd
import concurrent.futures
from yahooquery import search

# Load CSV file
file_path = "D://stocks//PITRO.csv"
df = pd.read_csv(file_path)

# Normalize column names (strip spaces, lowercase)
df.columns = df.columns.str.strip().str.lower().str.replace(" ", "_").str.replace("-", "_")

# Print column names to verify
print("Columns in dataset:", df.columns)

# Map correct column names
column_mapping = {
    "piotroski_score": "f_score",
    "altman_z_score": "z_score",
    "debt_to_equity": "debt_equity",
    "return_on_capital_employed": "roce",
    "return_on_equity": "roe",
    "sales_growth_3years": "growth",
    "current_ratio": "current_ratio"
}

# Rename columns
df = df.rename(columns=column_mapping)

# Check for missing columns
required_columns = list(column_mapping.values()) + ["name"]  # Include company name
missing_columns = [col for col in required_columns if col not in df.columns]
if missing_columns:
    print(f"Missing columns: {missing_columns}")
    raise KeyError(f"Missing required columns: {missing_columns}")

# Convert numeric columns to float
for col in ["f_score", "z_score", "debt_equity", "roce", "roe", "growth", "current_ratio"]:
    df[col] = pd.to_numeric(df[col], errors='coerce')

# Function to get ticker from company name
def get_ticker_yahoo(company_name):
    try:
        results = search(company_name)
        if "quotes" in results and results["quotes"]:
            return results["quotes"][0]["symbol"]
    except Exception as e:
        print(f"Error fetching ticker for {company_name}: {e}")
    return None

# Fetch tickers using threading for efficiency
with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:
    tickers = list(executor.map(get_ticker_yahoo, df["name"]))

# Add tickers to dataframe
df["ticker"] = tickers

# Apply filtering criteria
filtered_df = df[
    (df["f_score"] >= 7) & 
    (df["z_score"] > 4) & 
    (df["debt_equity"] < 1)
].copy()

# Normalize values for scoring
filtered_df["debt_equity_norm"] = 1 - (filtered_df["debt_equity"] / filtered_df["debt_equity"].max())  # Lower is better
filtered_df["current_ratio_norm"] = filtered_df["current_ratio"] / filtered_df["current_ratio"].max()
filtered_df["growth_norm"] = filtered_df["growth"] / filtered_df["growth"].max()
filtered_df["roce_norm"] = filtered_df["roce"] / filtered_df["roce"].max()
filtered_df["roe_norm"] = filtered_df["roe"] / filtered_df["roe"].max()

# Assign weightages
weights = {
    "debt_equity_norm": 0.15,
    "current_ratio_norm": 0.20,
    "growth_norm": 0.25,
    "roce_norm": 0.20,
    "roe_norm": 0.20
}

# Compute final weighted score
filtered_df["final_score"] = (
    filtered_df["debt_equity_norm"] * weights["debt_equity_norm"] +
    filtered_df["current_ratio_norm"] * weights["current_ratio_norm"] +
    filtered_df["growth_norm"] * weights["growth_norm"] +
    filtered_df["roce_norm"] * weights["roce_norm"] +
    filtered_df["roe_norm"] * weights["roe_norm"]
)

# Rank and get top 50 stocks
ranked_df = filtered_df.sort_values(by="final_score", ascending=False).head(50)

# Display result
print(ranked_df[["name", "ticker", "f_score", "z_score", "debt_equity", "current_ratio", "roce", "roe", "growth", "final_score"]])

# Save to CSV
output_path = "top_50_stocks_with_ticker.csv"
ranked_df.to_csv(output_path, index=False)
print(f"Top 50 stocks saved as '{output_path}'")

Columns in dataset: Index(['name', 'bse_code', 'nse_code', 'industry', 'current_price',
       'price_to_earning', 'market_capitalization',
       'return_on_capital_employed', 'eps_last_year', 'return_on_equity',
       'book_value', 'price_to_book_value', 'graham_number',
       'historical_pe_7years', 'current_ratio', 'peg_ratio', 'debt_to_equity',
       'altman_z_score', 'piotroski_score', 'npm_last_year',
       'sales_growth_3years', 'eps', 'industry_pe', 'quick_ratio',
       'debtor_days', 'number_of_equity_shares', 'intrinsic_value',
       'enterprise_value', 'return_on_invested_capital'],
      dtype='object')
                name           ticker  f_score  z_score  debt_equity  \
59    Mefcom Capital     MEFCOMCAP.BO      8.0    15.83         0.14   
87    Sumedha Fiscal       SUMEDHA.BO      8.0     5.53         0.00   
23      Danlaw Tech.        DANLAW.BO      9.0     8.56         0.56   
10   Arrow Greentech    ARROWGREEN.NS      8.0    11.62         0.01   
75  R S So

In [9]:
ranked_df[["name", "ticker", "f_score", "z_score", "debt_equity", "current_ratio", "roce", "roe", "growth", "final_score"]].head(70)

,name,ticker,f_score,z_score,debt_equity,current_ratio,roce,roe,growth,final_score
59,Mefcom Capital,MEFCOMCAP.BO,8.0,15.83,0.14,6.57,41.14,43.29,96.98,0.728401
87,Sumedha Fiscal,SUMEDHA.BO,8.0,5.53,0.00,19.93,15.56,12.96,68.54,0.625675
23,Danlaw Tech.,DANLAW.BO,9.0,8.56,0.56,2.37,40.21,47.46,96.49,0.603758
10,Arrow Greentech,ARROWGREEN.NS,8.0,11.62,0.01,10.71,35.32,26.07,41.52,0.575264
75,R S Software (I),RSSOFTWARE.NS,8.0,7.02,0.01,4.87,44.35,45.68,18.86,0.558063
91,Unique Organics,UNIQUEO.BO,8.0,10.63,0.07,4.01,38.07,32.63,43.41,0.532125
47,Kothari Petroche,KOTHARIPET.NS,8.0,8.25,0.01,3.94,41.32,28.79,38.54,0.529832
26,Dhanalaxmi Roto,DHANROTO.BO,8.0,6.29,0.11,2.69,38.40,28.43,39.23,0.485694
2,Action Const.Eq.,None,8.0,12.67,0.08,1.33,42.30,30.58,33.41,0.484622
22,Chandni Machines,CHANDNIMACH.BO,8.0,10.41,0.01,1.35,28.91,21.80,56.67,0.483357


In [12]:
import pandas as pd
import concurrent.futures
from yahooquery import search

# Load CSV file
file_path = "D://stocks//PITRO1.csv"
df = pd.read_csv(file_path)

# Normalize column names (strip spaces, lowercase)
df.columns = df.columns.str.strip().str.lower().str.replace(" ", "_").str.replace("-", "_")

# Print column names to verify
print("Columns in dataset:", df.columns)

# Map correct column names
column_mapping = {
    "piotroski_score": "f_score",
    "altman_z_score": "z_score",
    "debt_to_equity": "debt_equity",
    "return_on_capital_employed": "roce",
    "return_on_equity": "roe",
    "sales_growth_3years": "growth",
    "current_ratio": "current_ratio"
}

# Rename columns
df = df.rename(columns=column_mapping)

# Check for missing columns
required_columns = list(column_mapping.values()) + ["name"]  # Include company name
missing_columns = [col for col in required_columns if col not in df.columns]
if missing_columns:
    print(f"Missing columns: {missing_columns}")
    raise KeyError(f"Missing required columns: {missing_columns}")

# Convert numeric columns to float
for col in ["f_score", "z_score", "debt_equity", "roce", "roe", "growth", "current_ratio"]:
    df[col] = pd.to_numeric(df[col], errors='coerce')

# Function to get ticker from company name
def get_ticker_yahoo(company_name):
    try:
        results = search(company_name)
        if "quotes" in results and results["quotes"]:
            return results["quotes"][0]["symbol"]
    except Exception as e:
        print(f"Error fetching ticker for {company_name}: {e}")
    return None

# Fetch tickers using threading for efficiency
with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:
    tickers = list(executor.map(get_ticker_yahoo, df["name"]))

# Add tickers to dataframe
df["ticker"] = tickers

# Apply filtering criteria
filtered_df = df[
    (df["f_score"] >= 7) & 
    (df["z_score"] > 4) & 
    (df["debt_equity"] < 1)
].copy()

# Normalize values for scoring
filtered_df["debt_equity_norm"] = 1 - (filtered_df["debt_equity"] / filtered_df["debt_equity"].max())  # Lower is better
filtered_df["current_ratio_norm"] = filtered_df["current_ratio"] / filtered_df["current_ratio"].max()
filtered_df["growth_norm"] = filtered_df["growth"] / filtered_df["growth"].max()
filtered_df["roce_norm"] = filtered_df["roce"] / filtered_df["roce"].max()
filtered_df["roe_norm"] = filtered_df["roe"] / filtered_df["roe"].max()

# Assign weightages
weights = {
    "debt_equity_norm": 0.15,
    "current_ratio_norm": 0.20,
    "growth_norm": 0.25,
    "roce_norm": 0.20,
    "roe_norm": 0.20
}

# Compute final weighted score
filtered_df["final_score"] = (
    filtered_df["debt_equity_norm"] * weights["debt_equity_norm"] +
    filtered_df["current_ratio_norm"] * weights["current_ratio_norm"] +
    filtered_df["growth_norm"] * weights["growth_norm"] +
    filtered_df["roce_norm"] * weights["roce_norm"] +
    filtered_df["roe_norm"] * weights["roe_norm"]
)

# Rank and get top 50 stocks
ranked_df = filtered_df.sort_values(by="final_score", ascending=False).head(100)

# Display result
print(ranked_df[["name", "ticker", "f_score", "z_score", "debt_equity", "current_ratio", "roce", "roe", "growth", "final_score"]])

# Save to CSV
output_path = "top_100_stocks_with_ticker.csv"
ranked_df.to_csv(output_path, index=False)
print(f"Top 50 stocks saved as '{output_path}'")

Columns in dataset: Index(['name', 'bse_code', 'nse_code', 'industry', 'current_price',
       'price_to_earning', 'market_capitalization',
       'return_on_capital_employed', 'eps_last_year', 'return_on_equity',
       'book_value', 'price_to_book_value', 'graham_number',
       'historical_pe_7years', 'current_ratio', 'peg_ratio', 'debt_to_equity',
       'altman_z_score', 'piotroski_score', 'npm_last_year',
       'sales_growth_3years', 'eps', 'industry_pe', 'quick_ratio',
       'debtor_days', 'number_of_equity_shares', 'intrinsic_value',
       'enterprise_value', 'pledged_percentage', 'promoter_holding',
       'earnings_yield', 'free_cash_flow_3years', 'g_factor',
       'return_on_assets'],
      dtype='object')
                 name         ticker  f_score  z_score  debt_equity  \
107   Meghna Infracon        MIIL.BO      8.0    96.80         0.00   
74     Infronics Sys.   INFRONICS.BO      8.0    20.78         0.01   
112      Nestle India   NESTLEIND.NS      8.0    33.93  

In [14]:
ranked_df[["name", "ticker", "f_score", "z_score", "debt_equity", "current_ratio", "roce", "roe", "growth", "final_score"]].head(50)

,name,ticker,f_score,z_score,debt_equity,current_ratio,roce,roe,growth,final_score
107,Meghna Infracon,MIIL.BO,8.0,96.80,0.00,36.02,21.05,27.32,214.23,0.638913
74,Infronics Sys.,INFRONICS.BO,8.0,20.78,0.01,17.44,123.82,259.79,-0.76,0.524023
112,Nestle India,NESTLEIND.NS,8.0,33.93,0.09,0.75,169.08,135.33,22.25,0.434712
113,Network People,NPST-SM.NS,9.0,40.73,0.00,2.99,83.46,63.69,102.59,0.417714
105,Mefcom Capital,MEFCOMCAP.BO,8.0,15.83,0.14,6.57,41.14,43.29,96.98,0.347445
55,Ganesh Housing,GANESHHOUC.NS,8.0,10.29,0.02,5.75,45.01,41.58,74.35,0.339964
78,Jagsonpal Fin.,JAGSONFI.BO,8.0,31.70,0.00,4.16,45.86,45.86,60.91,0.321949
25,Benares Hotels,BENARAS.BO,9.0,17.57,0.03,5.27,40.56,31.12,70.70,0.320762
12,Arrow Greentech,ARROWGREEN.NS,8.0,11.62,0.01,10.71,35.32,26.07,41.52,0.311423
35,Colgate-Palmoliv,CL,8.0,37.07,0.04,1.31,96.80,74.52,5.47,0.309791
